In [79]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [80]:
hs300 = pd.read_csv("hs300.csv")
hs300["hs300_rt"] = hs300["涨跌幅(%)"] / 100
hs300_constituent_stocks = pd.read_csv("hs300_constituent_stocks.csv")
hs300_constituent_stocks = pd.merge(hs300_constituent_stocks, hs300[['交易日期','hs300_rt']],on="交易日期",how='left')
hs300_constituent_stocks['excess_rt'] = hs300_constituent_stocks['涨跌幅（%）'] - hs300_constituent_stocks['hs300_rt']
hs300_constituent_stocks['excess_std'] = hs300_constituent_stocks.groupby("股票代码")['excess_rt'].transform(lambda x: x.rolling(60).std())

In [81]:
hs300_constituent_stocks["交易日期"] = pd.to_datetime(hs300_constituent_stocks["交易日期"])
hs300_constituent_stocks_after_2021 = hs300_constituent_stocks[hs300_constituent_stocks['交易日期']>=pd.to_datetime("2021-01-04")]

In [82]:
hs300_constituent_stocks_predict = pd.read_csv("hs300_constituent_stocks_predict.csv")
hs300_constituent_stocks_predict["date"] = pd.to_datetime(hs300_constituent_stocks_predict["date"])
hs300_constituent_stocks_predict_filtered = hs300_constituent_stocks_predict[hs300_constituent_stocks_predict['date']<=pd.to_datetime("2023-12-29")]

In [83]:
df = pd.merge(hs300_constituent_stocks_after_2021,hs300_constituent_stocks_predict_filtered,left_on=['交易日期','股票代码'],right_on=['date','code'],how='outer')
df

,交易日期,股票代码,开盘价,最高价,最低价,收盘价,成交量（手）,成交金额（千元）,总市值,流通股,涨跌幅（%）,权重,hs300_rt,excess_rt,excess_std,date,code,pred
0,2021-01-04,1,19.10,19.10,18.44,18.60,155421643.0,2.891682e+09,3.609501e+11,1.940575e+10,-0.038263,0.9102,0.0108,-0.049063,0.019485,2021-01-04,1.0,183.0
1,2021-01-04,2,28.40,28.40,27.52,27.78,146844133.0,4.079417e+09,3.227406e+11,9.717553e+09,-0.032056,0.9537,0.0108,-0.042856,0.014659,2021-01-04,2.0,279.0
2,2021-01-04,63,33.00,34.48,32.72,34.33,129647346.0,4.343080e+09,1.583792e+11,3.476254e+09,0.020208,0.4676,0.0108,0.009408,0.024844,2021-01-04,63.0,192.0
3,2021-01-04,66,19.25,20.89,19.18,20.70,175854423.0,3.572707e+09,6.061337e+10,2.927812e+09,0.090047,0.1834,0.0108,0.079247,0.027555,2021-01-04,66.0,259.0
4,2021-01-04,69,7.05,7.06,6.87,6.89,53350376.0,3.697312e+08,5.651036e+10,7.056298e+09,-0.028209,0.1425,0.0108,-0.039009,0.012240,2021-01-04,69.0,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218095,2023-12-29,688363,67.54,67.98,66.89,66.93,2537752.0,1.707659e+08,3.223872e+10,1.981782e+08,-0.015591,0.0800,0.0049,-0.020491,0.015207,2023-12-29,688363.0,113.0
218096,2023-12-29,688396,44.57,44.94,44.18,44.69,2923514.0,1.303507e+08,5.899491e+10,1.320092e+09,0.001344,0.1440,0.0049,-0.003556,0.014802,2023-12-29,688396.0,224.0
218097,2023-12-29,688561,40.19,40.43,39.65,40.09,3084376.0,1.233603e+08,2.746856e+10,4.636838e+08,0.002501,0.0720,0.0049,-0.002399,0.015340,2023-12-29,688561.0,135.0
218098,2023-12-29,688599,28.90,28.99,28.23,28.53,21452337.0,6.129329e+08,6.201171e+10,2.173562e+09,-0.019588,0.1700,0.0049,-0.024488,0.020102,2023-12-29,688599.0,16.0


In [84]:
# 创建超额收益率矩阵（行：日期，列：股票代码，值：excess_rt）
hs300_constituent_stocks['交易日期'] = pd.to_datetime(hs300_constituent_stocks['交易日期'] )
excess_returns_matrix = hs300_constituent_stocks[hs300_constituent_stocks['交易日期']>pd.to_datetime('2020-10-01')].pivot(index='交易日期', columns='股票代码', values='excess_rt')
excess_returns_matrix

股票代码,1,2,63,66,69,100,157,166,301,333,...,688126,688169,688187,688223,688303,688363,688396,688561,688599,688981
交易日期,,,,,,,,,,,,,,,,,,,,,
2020-10-09,-0.019741,-0.020043,-0.011941,0.005161,-0.007126,0.034885,0.005494,-0.001568,NaN,-0.009518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-12,0.017131,-0.020311,0.014038,0.011037,0.003179,-0.016433,0.006960,0.001123,NaN,-0.022261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-13,0.006763,-0.014603,0.021944,-0.012057,-0.010342,-0.013938,-0.007935,-0.015845,NaN,0.022518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-14,0.004732,-0.000903,-0.005711,-0.016368,-0.020350,-0.021050,0.024062,-0.006104,NaN,-0.001833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-15,0.034763,-0.001180,-0.013597,-0.019397,0.001700,-0.006199,-0.010886,0.001700,NaN,0.012996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25,-0.004187,-0.010807,0.002859,NaN,-0.003100,0.012245,0.006188,0.001456,0.025471,-0.004995,...,0.002807,NaN,0.001519,0.008847,0.005402,0.006958,0.001674,0.012687,-0.004616,-0.035588
2023-12-26,-0.002993,-0.005821,-0.020056,NaN,-0.009593,-0.000757,-0.002402,-0.004538,0.046800,0.009268,...,-0.014339,NaN,-0.001821,-0.005006,-0.000225,0.002910,-0.022612,-0.017404,0.007180,-0.015083
2023-12-27,-0.001302,-0.003500,-0.003500,NaN,0.003167,0.044723,0.008884,0.005674,-0.003500,0.011652,...,-0.015498,NaN,-0.000891,-0.001111,-0.016588,-0.012247,0.006990,-0.008200,-0.012605,-0.002343


In [85]:
# 定义优化函数
def optimize_portfolio_scipy(mu, Sigma):
    n = len(mu)
    mu = np.array(mu)
    Sigma = np.array(Sigma)

    def neg_sharpe_ratio(w):
        ret = np.dot(w, mu)
        vol = np.sqrt(np.dot(w.T, np.dot(Sigma, w)))
        return -ret / (vol + 1e-8)

    constraints = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    )
    bounds = tuple((0, 1) for _ in range(n))  # 不允许做空

    init_guess = np.array([1.0 / n] * n)

    result = minimize(neg_sharpe_ratio, init_guess, bounds=bounds, constraints=constraints)

    if result.success:
        return result.x
    else:
        print("Optimization failed:", result.message)
        return np.array([1.0 / n] * n)

In [86]:
dates =sorted(df['交易日期'].unique())

portfolio_excess_returns = []
stock_dict = {}
weight_dict = {}

for date in dates:
    daily_data = df[(df['date'] == date) & (~df['pred'].isna())].copy()
    daily_data = daily_data.sort_values('pred', ascending=True)
    daily_data['is_new_low'] = daily_data['excess_std'] < daily_data['excess_std'].cummin().shift(1)
    stock_dict[date] = daily_data.loc[daily_data['is_new_low'], '股票代码'].tolist()
    stock_dict[date].insert(0, daily_data.iloc[0]['股票代码'])
    stock_pool = stock_dict[date]
    
    weight_dict[date]={}
    sum_weight = 1
    total_stock_num = len(stock_pool)
    base_weight = 1.0 / total_stock_num
    assigned_weight = 0.0
    cols_to_drop = []
    
    end_idx = excess_returns_matrix.index.get_loc(date)
    start_idx = end_idx - 59
    window_data = excess_returns_matrix.iloc[start_idx:end_idx+1][stock_pool]  

    for col in window_data.columns:
        if window_data[col].isna().any():
            weight_dict[date][col] = base_weight
            assigned_weight += base_weight
            cols_to_drop.append(col)

    window_data.drop(columns=cols_to_drop, inplace=True)

    if len(window_data.columns) == 1:
        # 如果只剩一个股票，也分配base_weight（如果原来只有一只股票的话，也是100%权重）
        remaining_col = window_data.columns[0]
        weight_dict[date][remaining_col] = base_weight
        assigned_weight += base_weight
        continue
    elif len(window_data.columns) > 1:
        
        # 计算均值与协方差矩阵（添加正则化）
        mu = window_data.mean().values
        Sigma = window_data.cov().values
        
        opt_weights = optimize_portfolio_scipy(mu, Sigma)
        for i, col in enumerate(window_data.columns):
            weight_dict[date][col] = opt_weights[i] * (1.0 - assigned_weight)

In [87]:
# 创建字典以便快速查找股票的每日超额收益
excess_rt_dict = df.set_index(['交易日期', '股票代码'])['excess_rt'].to_dict()
rt_dict = df.set_index(['交易日期', '股票代码'])['涨跌幅（%）'].to_dict()

# 初始化组合超额收益列表
portfolio_excess_returns = []
portfolio_returns = []
hs300_returns =  df.groupby('交易日期')['hs300_rt'].first().tolist()[1:]
hs300_returns = np.array(hs300_returns)

# 遍历每个交易日（排除最后一个日期，因为没有下一天）
for i in range(len(dates) - 1):
    current_date = dates[i]
    next_date = dates[i+1]
    
    # 获取当日的权重配置
    weights = weight_dict.get(current_date, {})
    
    daily_excess_return = 0.0
    daily_return = 0.0
    for stock, weight in weights.items():
        # 获取下一天该股票的超额收益，若无则默认为0
        excess_rt = excess_rt_dict.get((next_date, stock), 0.0)
        rt = rt_dict.get((next_date,stock), 0.0)
        
        daily_excess_return += weight * excess_rt
        daily_return += weight * rt
    
    portfolio_excess_returns.append(daily_excess_return)
    portfolio_returns.append(daily_return)

# 转换为NumPy数组
excess_returns = np.array(portfolio_excess_returns)
returns = np.array(portfolio_returns)

# 超额年化夏普
mean_return = excess_returns.mean()
std_return = excess_returns.std()
sharpe_ratio = mean_return / std_return * np.sqrt(252)

# 计算最大回撤
cumulative_nav = np.cumprod(1 + returns)
peak = np.maximum.accumulate(cumulative_nav)
drawdown = (peak - cumulative_nav) / peak
max_drawdown = drawdown.max()

# 累计绝对收益
cumulative_returns = cumulative_nav[-1] - 1
hs300_cumulative_returns = np.cumprod(1 + hs300_returns)[-1] - 1
# 累计超额收益
cumulative_excess = cumulative_returns - hs300_cumulative_returns

print(f"累计绝对收益: {cumulative_returns:.4f}")
print(f"累计超额收益: {cumulative_excess:.4f}")
print(f"最大回撤: {max_drawdown:.4f}")
print(f"超额年化夏普比率: {sharpe_ratio:.4f}")

累计绝对收益: 10.5785
累计超额收益: 10.9264
最大回撤: 0.1909
超额年化夏普比率: 3.2014
